![](imgs/kodolamaczlogo.png)

# Przetwarzanie Big Data z użyciem Apache Spark

Autor notebooka: Jakub Nowacki.

## Więcej niż MapReduce

In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder \
    .appName("beyondMapReduce") \
    .getOrCreate()
sc = spark.sparkContext

## Akumulatory

Akumulatory to zmienne tylko do zapisu dla workerów, które służą do zliczania wystąpień.

In [2]:
acc = sc.accumulator(0)

In [3]:
rdd = sc.textFile("data/titus_andronicus.txt")

In [4]:
rdd.foreach(lambda x: acc.add(1))

In [5]:
# Wartość dostępna jest jako
acc.value

3255

In [12]:
acc.aid

0

In [6]:
# Akumulatory można opakowywać w funkję, jeżeli zachodzi taka potrzeba
def function_with_acc(x):
    acc.add(1)

In [15]:
# Akumulatory można opakowywać w funkję, jeżeli zachodzi taka potrzeba
c = 0
def function_with_acc(x):
    global c
    c += 1
    acc.add(1)

In [16]:
rdd.foreach(function_with_acc)

In [17]:
acc.value

16275

In [18]:
c

0

In [19]:
# Zainicjujmy kolejny akumulator
acc2 = sc.accumulator(0)

In [20]:
# ... i związaną funkcję
def split_and_countspaces(x):
    acc2.add(x.count(" "))
    return x.split()

In [21]:
words = rdd.flatMap(split_and_countspaces)

### Zadania 

* Jaka jest wartość `acc2`?
* Pobierz 15 wyrazów: jaka jest teraz wartość `acc2`?
* Pobierz jeszcze raz i sprawdź wartość.
* Użyj `words.cache()`, pobierz 15 wyrazów i sprawdź `acc2.value`.
* Weż pierwsze 15 wyrazów i sprawdź `acc2.value`.

In [22]:
acc2.value

0

In [23]:
words.take(15)

['This',
 'Etext',
 'file',
 'is',
 'presented',
 'by',
 'Project',
 'Gutenberg,',
 'in',
 'cooperation',
 'with',
 'World',
 'Library,',
 'Inc.,',
 'from']

In [24]:
acc2.value

17

In [25]:
words.take(15)

['This',
 'Etext',
 'file',
 'is',
 'presented',
 'by',
 'Project',
 'Gutenberg,',
 'in',
 'cooperation',
 'with',
 'World',
 'Library,',
 'Inc.,',
 'from']

In [26]:
acc2.value

34

In [27]:
words.cache()
words.take(15)

['This',
 'Etext',
 'file',
 'is',
 'presented',
 'by',
 'Project',
 'Gutenberg,',
 'in',
 'cooperation',
 'with',
 'World',
 'Library,',
 'Inc.,',
 'from']

In [28]:
acc2.value

16059

In [29]:
words.take(15)

['This',
 'Etext',
 'file',
 'is',
 'presented',
 'by',
 'Project',
 'Gutenberg,',
 'in',
 'cooperation',
 'with',
 'World',
 'Library,',
 'Inc.,',
 'from']

In [30]:
acc2.value

16059

## Broadcasts

Zmienne tylko do odczytu pozwalające przekazywać argumenty do zadań. 

In [31]:
# Zmiennych tych używa się do przekazywania argumentów; wartości te mogą być dość duże
br = sc.broadcast(["Marcus", "Titus", "hate", "die"])

In [32]:
br.value

['Marcus', 'Titus', 'hate', 'die']

In [33]:
def line_of_brothers(line):
    t = 0
    for word in br.value:
        if word in line:
            t += 1
    return t > 1

In [34]:
rdd.filter(line_of_brothers).take(5)

['    Let Marcus, Lucius, or thyself, old Titus,',
 "  TITUS. Marcus, my brother! 'Tis sad Titus calls."]

In [35]:
rdd.filter(line_of_brothers).takeSample(False, 5)

["  TITUS. Marcus, my brother! 'Tis sad Titus calls.",
 '    Let Marcus, Lucius, or thyself, old Titus,']

In [36]:
b = ["Marcus", "Titus", "hate", "die"]
def line_of_brothers_local(line):
    t = 0
    for word in b:
        if word in line:
            t += 1
    return t > 1

In [37]:
rdd.filter(line_of_brothers_local).take(5)

['    Let Marcus, Lucius, or thyself, old Titus,',
 "  TITUS. Marcus, my brother! 'Tis sad Titus calls."]

## Operacje na zbiorach

In [38]:
rdd1 = sc.parallelize(["cat", "dog", "python"])
rdd2 = sc.parallelize(["magpie", "python", "elephant"])

In [39]:
# Konkatenacja a nie suma zbiorów
rdd1.union(rdd2).collect()

['cat', 'dog', 'python', 'magpie', 'python', 'elephant']

### Zadanie

* Żeby otrzymać prawdziwą sumę zbiorów z `union` użyj `.distinct()`.

In [40]:
rdd1.union(rdd2).distinct().collect()

['cat', 'python', 'elephant', 'magpie', 'dog']

In [41]:
# Część wspólna zbiorów
rdd1.intersection(rdd2).collect()

['python']

## Złączenie zbiorów

Łączenie zbiorów (join) działa podobnie jak w SQL ale odbywa się na parze klucz wartość w formie `(key, value)`.

In [46]:
species = sc.parallelize([("Max", "dog"), ("Sasha", "cat"), ("Alice", "snake"), ("Ulrich", "snake")])
ages = sc.parallelize([("Alice", 3), ("Max", 11), ("Ulrich", 2), ("Karl", 31)])

In [47]:
# inner join
species.join(ages) \
  .collect()

[('Ulrich', ('snake', 2)), ('Max', ('dog', 11)), ('Alice', ('snake', 3))]

### Zadanie

* Wypisz zbiór wierząt ze znanym wiekiem.

In [50]:
species.join(ages)\
    .map(lambda t: t[1][0]) \
    .distinct() \
    .collect()

['snake', 'dog']

In [55]:
s = species.join(ages)\
    .map(lambda t: t[1][0]) \
    .collect()
frozenset(s)

frozenset({'dog', 'snake'})

In [57]:
species.join(ages)\
    .mapValues(lambda v: v[0]) \
    .collect()

[('Ulrich', 'snake'), ('Max', 'dog'), ('Alice', 'snake')]

In [44]:
# Podpowiedź
z = ("a", ("b", "c"))
print(z[1])
print(z[1][0])

('b', 'c')
b


In [58]:
# left outer join
species.leftOuterJoin(ages) \
  .collect()

[('Ulrich', ('snake', 2)),
 ('Max', ('dog', 11)),
 ('Sasha', ('cat', None)),
 ('Alice', ('snake', 3))]

In [59]:
# right outer join
species.rightOuterJoin(ages) \
  .collect()

[('Ulrich', ('snake', 2)),
 ('Karl', (None, 31)),
 ('Max', ('dog', 11)),
 ('Alice', ('snake', 3))]

In [60]:
# full outer join
species.fullOuterJoin(ages) \
  .collect()

[('Ulrich', ('snake', 2)),
 ('Karl', (None, 31)),
 ('Max', ('dog', 11)),
 ('Sasha', ('cat', None)),
 ('Alice', ('snake', 3))]

In [61]:
# iloczyn kartezjański
species.cartesian(ages) \
  .collect()

[(('Max', 'dog'), ('Alice', 3)),
 (('Max', 'dog'), ('Max', 11)),
 (('Sasha', 'cat'), ('Alice', 3)),
 (('Sasha', 'cat'), ('Max', 11)),
 (('Max', 'dog'), ('Ulrich', 2)),
 (('Max', 'dog'), ('Karl', 31)),
 (('Sasha', 'cat'), ('Ulrich', 2)),
 (('Sasha', 'cat'), ('Karl', 31)),
 (('Alice', 'snake'), ('Alice', 3)),
 (('Alice', 'snake'), ('Max', 11)),
 (('Ulrich', 'snake'), ('Alice', 3)),
 (('Ulrich', 'snake'), ('Max', 11)),
 (('Alice', 'snake'), ('Ulrich', 2)),
 (('Alice', 'snake'), ('Karl', 31)),
 (('Ulrich', 'snake'), ('Ulrich', 2)),
 (('Ulrich', 'snake'), ('Karl', 31))]

### Zadanie

* ★ Zamień produkt kartezjański w wewnętrzne złączenie (inner join).
* ★ Wypisz wszystkie możliwe pary zwierząt, np. `[("Max", "Alice"), ... ("Ulrich", "Karl")]`.

## Grupowanie (po kluczu)

In [62]:
cities = sc.parallelize(["Paris, France", "Lyon, France", "Warsaw, Poland", "London, United Kingdom"])

In [64]:
cities \
  .groupBy(lambda x: x.split(", ")[1]) \
  .map(lambda x: (x[0], list(x[1]))) \
  .collect()

[('France', ['Paris, France', 'Lyon, France']),
 ('Poland', ['Warsaw, Poland']),
 ('United Kingdom', ['London, United Kingdom'])]

In [68]:
cities \
  .groupBy(lambda x: x.split(", ")[1]) \
  .mapValues(lambda x: list(x)) \
  .collect()

[('France', ['Paris, France', 'Lyon, France']),
 ('Poland', ['Warsaw, Poland']),
 ('United Kingdom', ['London, United Kingdom'])]

### Zadanie

* Zamiast `.map(lambda x: (x[0], list(x[1])))` użyj `.mapValues(some_function)`. 

In [65]:
# Podpowiedź
sc.parallelize([("a", 3), ("b", -2)]) \
  .mapValues(lambda x: x*x) \
  .collect()

[('a', 9), ('b', 4)]

### Więcej zadań

Wracamy do Titus Andronicus:

* Wyczyść dane przez usuniecie niechcianych znaków
* Policz i wyświetl 20 najczęstrzych:
    * słów zaczynających się zawsze wielką literą,
    * słów nigdy nie zaczynających się od wielkiej litery.
* Policz częstotliwość (i.e. $p_{word}=n_{word}/n$ wszystkich słów (ignorując wielkość liter).
* Pobierz słownik z [tej strony](http://www.math.sjsu.edu/~foster/dictionary.txt); policz ile słów ze słownika występuje w dziele (słownik ma tylko wyrazy o małych literach).
* ★ Pobierz dzieło nie napisane przez Shakespeare [Gutenberg Project](http://www.gutenberg.org/); znajdź słowa charakterystyczne dla Shakespeare'a, tj. które nie występują w innym dziele.
* ★ Dla wszystkich słów w lini policz

$$\frac{p_{pair}}{p_{word1}p_{word2}} = \frac{n_{pair} n}{n_{word1}n_{word2}},$$

tylko dla par występujących więcej niż 5 razy; pokaż 20 najwyższych wartości.

In [73]:
import re
words2 = rdd.flatMap(lambda l: re.findall('\w+', l))\
    .map(lambda w: (w, 1))
words2.take(10)

[('This', 1),
 ('Etext', 1),
 ('file', 1),
 ('is', 1),
 ('presented', 1),
 ('by', 1),
 ('Project', 1),
 ('Gutenberg', 1),
 ('in', 1),
 ('cooperation', 1)]

In [82]:
from operator import add
words2.filter(lambda t: t[0][0].isupper() and (len(t[0]) == 1 or t[0][1:].islower()))\
    .reduceByKey(add)\
    .mapValues(lambda v: v/n) \
    .top(20, key=lambda t: t[1])

[('I', 0.01826672728024135),
 ('And', 0.012728850683969088),
 ('Rome', 0.004835310162416829),
 ('To', 0.00421539860313262),
 ('The', 0.003760796792990867),
 ('That', 0.003554159606562797),
 ('O', 0.0028929206099929743),
 ('But', 0.0027689382981361324),
 ('Emperor', 0.0024383187998512214),
 ('For', 0.002396991362565607),
 ('What', 0.0023143364879943795),
 ('My', 0.0021490267388519237),
 ('Andronicus', 0.002066371864280696),
 ('This', 0.002066371864280696),
 ('A', 0.002066371864280696),
 ('Lucius', 0.002066371864280696),
 ('Titus', 0.0018597346778526263),
 ('Lavinia', 0.0016944249287101708),
 ('Marcus', 0.0016944249287101708),
 ('Enter', 0.001611770054138943)]

In [81]:
n = words2.count()
n

24197

In [83]:
eng = sc.textFile('data/dictionary.txt')
eng.take(10)

['aaaa',
 'aaaaa',
 'aaaaaa',
 'aaaaaaa',
 'aaaaaaaa',
 'aaaaaaaah',
 'aaaaaaauugh',
 'aaaaaagh',
 'aaaaaahhhhh',
 'aaaaaaugh']

In [85]:
words2.map(lambda t: t[0].lower()).intersection(eng).take(10)

['gutenberg',
 'cooperation',
 'library',
 'cdroms',
 'placed',
 'copyright',
 'machine',
 'long',
 'others',
 'only']

In [87]:
r = words2.map(lambda t: (t[0].lower(), t[1]))
r.take(10)

[('this', 1),
 ('etext', 1),
 ('file', 1),
 ('is', 1),
 ('presented', 1),
 ('by', 1),
 ('project', 1),
 ('gutenberg', 1),
 ('in', 1),
 ('cooperation', 1)]

In [90]:
l = eng.map(lambda w: (w, None))
l.take(10)

[('aaaa', None),
 ('aaaaa', None),
 ('aaaaaa', None),
 ('aaaaaaa', None),
 ('aaaaaaaa', None),
 ('aaaaaaaah', None),
 ('aaaaaaauugh', None),
 ('aaaaaagh', None),
 ('aaaaaahhhhh', None),
 ('aaaaaaugh', None)]

In [92]:
r.join(l).mapValues(lambda t: t[0]).reduceByKey(add).top(10, key=lambda t: t[1])

[('that', 300),
 ('with', 284),
 ('this', 248),
 ('titus', 196),
 ('thou', 174),
 ('your', 140),
 ('will', 137),
 ('have', 133),
 ('marcus', 124),
 ('lucius', 120)]